# Self-Driving Car Engineer Nanodegree


## Project: **Behavioral Cloning** 
***

This is an extension to the previous notebook P3.ipynb. In this i use a fit generator.
I don't do any data exploration here. Instead this is essentially what feeds into my final "model.py"

---

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import math
import os
import csv
import random
from scipy import signal
import tensorflow as tf
from glob import glob
import sklearn

from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Lambda
from keras.layers import Cropping2D
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop

%matplotlib inline

Using TensorFlow backend.


## Read in Images

In [2]:
lines = []

with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

#print(lines[0])
#print(lines[1])
#print(lines[2:4])

# This is required since the first line is comprised of headers.
lines = lines[1:]

train_samples, validation_samples = train_test_split(lines, test_size=0.2)

print(len(train_samples))
print(len(validation_samples))

6428
1608


In [3]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = '../data/IMG/'+batch_sample[0].split('/')[-1]
                image = cv2.imread(name)
                center_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

                image_flipped = np.copy(np.fliplr(image))
                image_flipped_rgb = cv2.cvtColor(image_flipped, cv2.COLOR_BGR2RGB)
                images.append(image_flipped_rgb)
                angle_flipped = -center_angle
                angles.append(angle_flipped)
                
                name = '../data/IMG/'+batch_sample[1].split('/')[-1]
                image = cv2.imread(name)
                left_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                left_angle = center_angle + 0.065
                images.append(left_image)
                angles.append(left_angle)
                
                name = '../data/IMG/'+batch_sample[2].split('/')[-1]
                image = cv2.imread(name)
                right_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                right_angle = center_angle - 0.065
                images.append(right_image)
                angles.append(right_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

## Setup Generators

In [4]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

## Nvidia Deep Learning Model

In [5]:
model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Conv2D(24, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(36, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(48, 5, 5, subsample=(2,2), activation = 'relu'))
model.add(Conv2D(64, 3, 3, subsample=(1,1), activation = 'relu'))
model.add(Conv2D(64, 3, 3, subsample=(1,1), activation = 'relu'))

model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.4))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 65, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 158, 24)   1824        cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 77, 36)    21636       convolution2d_1[0][0]            
___________________________________________________________________________________________

## Compile The Model

In [8]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer='adam', loss='mse')

## Train The Model

In [9]:
epochs = 2

history_object = model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=epochs)

Epoch 1/2
6400/6428 [============================>.] - ETA: 0s - loss: 0.0112

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6528/6428 [==============================] - 13s - loss: 0.0111 - val_loss: 0.0097
Epoch 2/2
6512/6428 [==============================] - 13s - loss: 0.0106 - val_loss: 0.0103


## Save the Model

In [10]:
model.save('model.h5')

In [ ]:
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()